In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import util
import time
import os 
import itertools
from statsmodels.stats.multitest import multipletests
from scipy.stats import pearsonr

### Define functions

In [2]:
#SEBASTIEN URCHS
def p_permut(empirical_value, permutation_values):
    n_permutation = len(permutation_values)
    if empirical_value >= 0:
        return (np.sum(permutation_values > empirical_value)+1) / (n_permutation + 1)
    return (np.sum(permutation_values < empirical_value)+1) / (n_permutation + 1)

def filter_fdr(df,contrasts):
    df_filtered = df[(df['pair0'].isin(contrasts)) | (df['pair1'].isin(contrasts))]
    _,fdr,_,_ = multipletests(df_filtered['pval'],method='fdr_bh')
    df_filtered['fdr_filtered'] = fdr
    return df_filtered

def mat_form(df,contrasts,value = 'betamap_corr'):
    n = len(contrasts)
    d = dict(zip(contrasts,range(n)))
    mat = np.zeros((n,n))
    for c in contrasts:
        #fill out vertical strip of mat
        for i in range(n):
            if (i == d[c]):
                val = 1
            else:
                val = df[((df['pair0']==c)|(df['pair1']==c))
                                & ((df['pair0']==contrasts[i])|(df['pair1']==contrasts[i]))][value]
            mat[i,d[c]] = val
            mat[d[c],i] = val
    return pd.DataFrame(mat,columns=contrasts,index=contrasts)

def make_matrices(df,contrasts,fdr = 'fdr_filtered'):
    "Param fdr can be set to 'fdr_filtered': FDR is performed using the pvalues only from the chosen contrasts"
    "                              or 'fdr': values taken from FDR performed on full set of 42 contrasts"
    if (fdr == 'fdr_filtered'):
        df = filter_fdr(df,contrasts)
    mat_corr = mat_form(df,contrasts,value = 'betamap_corr')
    mat_pval = mat_form(df,contrasts,value = 'pval')
    mat_fdr = mat_form(df,contrasts,value = fdr)
    return mat_corr,mat_pval,mat_fdr

### Define labels

In [3]:
cases = ['DEL15q11_2','DUP15q11_2','DUP15q13_3_CHRNA7','DEL2q13','DUP2q13','DEL16p13_11','DUP16p13_11','DEL13q12_12','DUP13q12_12',
        'DEL17p12','DUP17p12','TAR_dup','NRXN1del','DEL1q21_1','DUP1q21_1','DEL22q11_2','DUP22q11_2','DEL16p11_2','DUP16p11_2',
      'SZ','BIP','ASD','ADHD']
prs = ['Stand_PRS_newCDG2_ukbb','Stand_PRS_ASD','Stand_PRS_SCZ','Stand_PRS_MDD','Stand_PRS_IQ', 'Stand_PRS_BMI','Stand_PRS_height',
       'Stand_PRS_T2D','Stand_PRS_LDL','Stand_PRS_CKD','Stand_PRS_SA','Stand_PRS_thickness','Stand_PRS_IBD_ukbb']
cont = prs + ['CT','SA','Vol','fluid_intelligence_score_all','Gfactor','Neuroticism']

maps = cases + cont

### Define paths
Only interested in mean corrected betamaps for correlations.

In [4]:
n_path_mc = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/null_models/{}_null_model_mc.npy'
cont_n_path_mc = '/home/harveyaa/Documents/fMRI/data/ukbb_9cohorts/null_models_continuous/{}_null_model_mc.npy'

b_path_mc = '/home/harveyaa/Documents/clara_paper/drop_maillard_15q11_2del/cc_{}_results_mc.csv'
cont_b_path_mc = '/home/harveyaa/Documents/clara_paper/drop_maillard_15q11_2del/cont_{}_results_mc.csv'

### Load null models and betamaps

In [5]:
null = []
beta = []
beta_std = []
for c in cases:
    null.append(pd.DataFrame(np.load(n_path_mc.format(c))))
    beta.append(pd.read_csv(b_path_mc.format(c))['betas'].values) #unstandardized betas
    beta_std.append(pd.read_csv(b_path_mc.format(c))['betas_std'].values) #standardized betas
    

for c in cont:
    null.append(pd.DataFrame(np.load(cont_n_path_mc.format(c))))
    if c not in prs:
        c = '{}_z'.format(c)
    beta.append(pd.read_csv(cont_b_path_mc.format(c))['betas'].values) #unstandardized betas
    beta_std.append(pd.read_csv(cont_b_path_mc.format(c))['betas_std'].values) #unstandardized betas

In [6]:
betamaps = pd.DataFrame(beta,index=maps)
betamaps_std = pd.DataFrame(beta_std,index=maps)
nullmodels = pd.concat(null,keys=maps)

# **Find pvalues**
- For whole connectome, and filtered to THAL and MOTnetDL:
  - For each pair of contrasts:
    - Generate a null distribution of correlations
    - Get the actual correlation values
    - Compare the actual correlation to the distribution to get the pvalue
    
### **Whole connectome**

### Get correlation distributions
For each unique pair, between the real map of one and the null maps of the other and vice versa. Use unstandardized betas here (needs to match unstandardized betas of null).

In [7]:
n_pairs = int((len(maps))*(len(maps) -1)/2)
corr = np.zeros((n_pairs,10000))

pair = []
l = 0
for i in itertools.combinations(maps,2):
    b = i[0]
    n = i[1]
    for j in range(5000):
        corr[l,j] = pearsonr(betamaps.loc[b].values,nullmodels.loc[n].values[j,:])[0]
        
    b = i[1]
    n = i[0]
    for j in range(5000):
        corr[l,j+5000] = pearsonr(betamaps.loc[b].values,nullmodels.loc[n].values[j,:])[0]
    
    pair.append(i)
    if (l%50 == 0):
        print('{}/{}'.format(l,n_pairs))
    l = l + 1

0/861
50/861
100/861
150/861
200/861
250/861
300/861
350/861
400/861
450/861
500/861
550/861
600/861
650/861
700/861
750/861
800/861
850/861


In [8]:
df = pd.DataFrame(corr)
df['pair'] = pair
df.to_csv('correlation_dist.csv')

### Get actual correlations
For each unique pair, correlation between betamaps. Use standardized betas here (as in rest of paper).

In [9]:
corr_bb = np.zeros(n_pairs)

pair_bb = []
l = 0
for i in itertools.combinations(maps,2):
    corr_bb[l] = pearsonr(betamaps_std.loc[i[0]].values,betamaps_std.loc[i[1]].values)[0]
    
    l = l + 1
    pair_bb.append(i)

In [10]:
df_bb = pd.DataFrame(corr_bb)
df_bb['pair'] = pair_bb
df_bb.to_csv('correlation_betas.csv')

### Merge DataFrames

In [11]:
(df['pair'] == df_bb['pair']).sum()

861

In [12]:
df_bb = df_bb.rename(columns={0:'betamap_corr'})

In [13]:
df_master = df_bb.merge(df,on='pair')

### Calculate pvals

In [14]:
pval = []
for i in df_master.index:
    p = p_permut(df_master.loc[i,'betamap_corr'],df_master[range(10000)].loc[i])
    pval.append(p)

In [15]:
df_master['pval'] = pval

### Add labels

In [16]:
pair0 = [p[0] for p in pair]
pair1 = [p[1] for p in pair]
df_master['pair0'] = pair0
df_master['pair1'] = pair1

In [17]:
df_compact = df_master[['pair0','pair1','betamap_corr','pval']]
df_compact.to_csv('corr_pval.csv')

### **Filtered by region**

In [18]:
mask = np.tri(64,k=0,dtype=bool)
        
THAL = np.zeros((64,64),bool)
THAL[:,3] = True
THAL_mask = THAL + np.transpose(THAL)
THAL_mask = np.tril(THAL_mask)
THAL_mask = THAL_mask[mask]
        
MOTnet_dl = np.zeros((64,64),bool)
MOTnet_dl[:,55] = True
MOTnet_dl_mask = MOTnet_dl + np.transpose(MOTnet_dl)
MOTnet_dl_mask = np.tril(MOTnet_dl_mask)
MOTnet_dl_mask = MOTnet_dl_mask[mask]

### **Thalamus**

### Filter maps
Filter betamaps and null models to only thalamus (region).

In [19]:
null_THAL = [n.transpose()[THAL_mask].transpose() for n in null]
beta_THAL = [b[THAL_mask] for b in beta]
beta_std_THAL = [b[THAL_mask] for b in beta_std]

betamaps_THAL = pd.DataFrame(beta_THAL,index=maps)
betamaps_std_THAL = pd.DataFrame(beta_std_THAL,index=maps)
nullmodels_THAL = pd.concat(null_THAL,keys=maps)

### Get correlation distributions
For each unique pair, between the real map of one and the null maps of the other and vice versa. Use unstandardized betas here (needs to match unstandardized betas of null).

In [20]:
n_pairs = int((len(maps))*(len(maps) -1)/2)
corr_THAL = np.zeros((n_pairs,10000))

pair_THAL = []
l = 0
for i in itertools.combinations(maps,2):
    b = i[0]
    n = i[1]
    for j in range(5000):
        corr_THAL[l,j] = pearsonr(betamaps_THAL.loc[b].values,nullmodels_THAL.loc[n].values[j,:])[0]
        
    b = i[1]
    n = i[0]
    for j in range(5000):
        corr_THAL[l,j+5000] = pearsonr(betamaps_THAL.loc[b].values,nullmodels_THAL.loc[n].values[j,:])[0]
    
    pair_THAL.append(i)
    if (l%50 == 0):
        print('{}/{}'.format(l,n_pairs))
    l = l + 1

0/861
50/861
100/861
150/861
200/861
250/861
300/861
350/861
400/861
450/861
500/861
550/861
600/861
650/861
700/861
750/861
800/861
850/861


In [21]:
df_THAL = pd.DataFrame(corr_THAL)
df_THAL['pair'] = pair_THAL
df_THAL.to_csv('correlation_dist_THAL.csv')

### Get actual correlations
For each unique pair, correlation between betamaps. Use standardized betas here (as in rest of paper).

In [22]:
corr_bb_THAL = np.zeros(n_pairs)

pair_bb_THAL = []
l = 0
for i in itertools.combinations(maps,2):
    corr_bb_THAL[l] = pearsonr(betamaps_std_THAL.loc[i[0]].values,betamaps_std_THAL.loc[i[1]].values)[0]
    
    l = l + 1
    pair_bb_THAL.append(i)

In [23]:
df_bb_THAL = pd.DataFrame(corr_bb_THAL)
df_bb_THAL['pair'] = pair_bb_THAL
df_bb_THAL.to_csv('correlation_betas_THAL.csv')

### Merge DataFrames

In [24]:
(df_THAL['pair'] == df_bb_THAL['pair']).sum()

861

In [25]:
df_bb_THAL = df_bb_THAL.rename(columns={0:'betamap_corr'})

In [26]:
df_master_THAL = df_bb_THAL.merge(df_THAL,on='pair')

### Calculate pvals

In [27]:
pval_THAL = []
for i in df_master_THAL.index:
    p = p_permut(df_master_THAL.loc[i,'betamap_corr'],df_master_THAL[range(10000)].loc[i])
    pval_THAL.append(p)

In [28]:
df_master_THAL['pval'] = pval_THAL

### Add labels

In [29]:
pair0_THAL = [p[0] for p in pair_THAL]
pair1_THAL = [p[1] for p in pair_THAL]
df_master_THAL['pair0'] = pair0_THAL
df_master_THAL['pair1'] = pair1_THAL

In [30]:
df_compact_THAL = df_master_THAL[['pair0','pair1','betamap_corr','pval']]
df_compact_THAL.to_csv('corr_pval_THAL.csv')

### **MOTnetDL**

### Filter maps
Filter betamaps and null models to only thalamus (region).

In [31]:
null_MOT = [n.transpose()[MOTnet_dl_mask].transpose() for n in null]
beta_MOT = [b[MOTnet_dl_mask] for b in beta]
beta_std_MOT = [b[MOTnet_dl_mask] for b in beta_std]

betamaps_MOT = pd.DataFrame(beta_MOT,index=maps)
betamaps_std_MOT = pd.DataFrame(beta_std_MOT,index=maps)
nullmodels_MOT = pd.concat(null_MOT,keys=maps)

### Get correlation distributions
For each unique pair, between the real map of one and the null maps of the other and vice versa. Use unstandardized betas here (needs to match unstandardized betas of null).

In [32]:
n_pairs = int((len(maps))*(len(maps) -1)/2)
corr_MOT = np.zeros((n_pairs,10000))

pair_MOT = []
l = 0
for i in itertools.combinations(maps,2):
    b = i[0]
    n = i[1]
    for j in range(5000):
        corr_MOT[l,j] = pearsonr(betamaps_MOT.loc[b].values,nullmodels_MOT.loc[n].values[j,:])[0]
        
    b = i[1]
    n = i[0]
    for j in range(5000):
        corr_MOT[l,j+5000] = pearsonr(betamaps_MOT.loc[b].values,nullmodels_MOT.loc[n].values[j,:])[0]
    
    pair_MOT.append(i)
    if (l%50 == 0):
        print('{}/{}'.format(l,n_pairs))
    l = l + 1

0/861
50/861
100/861
150/861
200/861
250/861
300/861
350/861
400/861
450/861
500/861
550/861
600/861
650/861
700/861
750/861
800/861
850/861


In [33]:
df_MOT = pd.DataFrame(corr_MOT)
df_MOT['pair'] = pair_MOT
df_MOT.to_csv('correlation_dist_MOT.csv')

### Get actual correlations
For each unique pair, correlation between betamaps. Use standardized betas here (as in rest of paper).

In [34]:
corr_bb_MOT = np.zeros(n_pairs)

pair_bb_MOT = []
l = 0
for i in itertools.combinations(maps,2):
    corr_bb_MOT[l] = pearsonr(betamaps_std_MOT.loc[i[0]].values,betamaps_std_MOT.loc[i[1]].values)[0]
    
    l = l + 1
    pair_bb_MOT.append(i)

In [35]:
df_bb_MOT = pd.DataFrame(corr_bb_MOT)
df_bb_MOT['pair'] = pair_bb_MOT
df_bb_MOT.to_csv('correlation_betas_MOT.csv')

### Merge DataFrames

In [36]:
(df_MOT['pair'] == df_bb_MOT['pair']).sum()

861

In [37]:
df_bb_MOT = df_bb_MOT.rename(columns={0:'betamap_corr'})

In [38]:
df_master_MOT = df_bb_MOT.merge(df_MOT,on='pair')

### Calculate pvals

In [39]:
pval_MOT = []
for i in df_master_MOT.index:
    p = p_permut(df_master_MOT.loc[i,'betamap_corr'],df_master_MOT[range(10000)].loc[i])
    pval_MOT.append(p)

In [40]:
df_master_MOT['pval'] = pval_MOT

### Add labels

In [41]:
pair0_MOT = [p[0] for p in pair_MOT]
pair1_MOT = [p[1] for p in pair_MOT]
df_master_MOT['pair0'] = pair0_MOT
df_master_MOT['pair1'] = pair1_MOT

In [42]:
df_compact_MOT = df_master_MOT[['pair0','pair1','betamap_corr','pval']]
df_compact_MOT.to_csv('corr_pval_MOT.csv')

In [43]:
## Make matrices

### Load DataFrame

In [64]:
df = pd.read_csv('corr_pval.csv',index_col=0)
df_THAL = pd.read_csv('corr_pval_THAL.csv',index_col=0)
df_MOT = pd.read_csv('corr_pval_MOT.csv',index_col=0)

### Define labels

In [65]:
cases = ['DEL15q11_2','DUP15q11_2','DUP15q13_3_CHRNA7','DEL2q13','DUP2q13','DEL16p13_11','DUP16p13_11','DEL13q12_12','DUP13q12_12',
        'DEL17p12','DUP17p12','TAR_dup','NRXN1del','DEL1q21_1','DUP1q21_1','DEL22q11_2','DUP22q11_2','DEL16p11_2','DUP16p11_2',
      'SZ','BIP','ASD','ADHD']
prs = ['Stand_PRS_newCDG2_ukbb','Stand_PRS_ASD','Stand_PRS_SCZ','Stand_PRS_MDD','Stand_PRS_IQ', 'Stand_PRS_BMI','Stand_PRS_height',
       'Stand_PRS_T2D','Stand_PRS_LDL','Stand_PRS_CKD','Stand_PRS_SA','Stand_PRS_thickness','Stand_PRS_IBD_ukbb']
cont = prs + ['CT','SA','Vol','fluid_intelligence_score_all','Gfactor','Neuroticism']

contrasts = cont + cases

# **Make matrices**
 - Turn the large dataframe with one entry for each pair into readable correlation matrix form
   - Use a specified subset of the contrasts
 - Return three matrices:
   - Correlation between betamaps
   - pvalues (from permutation test)
   - FDR corrected pvalues
     - FDR using given set of contrasts

## **Whole connectome**

In [66]:
#Example subset from figure 4
subset = ['DEL1q21_1','DUP22q11_2','DEL22q11_2','Stand_PRS_ASD','BIP','SZ','Neuroticism',
          'Stand_PRS_MDD','ASD','Stand_PRS_SCZ','DEL15q11_2','DUP16p11_2','DEL16p11_2',
          'DUP1q21_1','Stand_PRS_SA','SA','DUP17p12','CT','Gfactor','fluid_intelligence_score_all','Stand_PRS_IQ']

corr,pval,fdr = make_matrices(df,subset,fdr='fdr_filtered')

<ipython-input-2-91cc07d6591e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['fdr_filtered'] = fdr


In [67]:
corr.to_csv('FC_corr_fig4_wholebrain_mc.csv')
pval.to_csv('FC_corr_pval_fig4_wholebrain_mc.csv')
fdr.to_csv('FC_corr_fdr_filtered_fig4_wholebrain_mc.csv')
#fdr.to_csv('FC_corr_fdr_BY_filtered_fig4_wholebrain_mc.csv')

## **Thalamus**

In [68]:
#Example subset from figure 5
subset5 = ['CT','Gfactor','fluid_intelligence_score_all','Stand_PRS_IQ','Stand_PRS_SA',
           'DUP17p12','SA','Stand_PRS_ASD','DUP16p11_2','DEL1q21_1','DUP22q11_2','DEL16p11_2',
           'DUP1q21_1','DEL22q11_2','BIP','SZ','Neuroticism','ASD','DEL15q11_2',
          'Stand_PRS_SCZ','Stand_PRS_MDD']

corr_THAL,pval_THAL,fdr_THAL = make_matrices(df_THAL,subset5,fdr='fdr_filtered')

<ipython-input-2-91cc07d6591e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['fdr_filtered'] = fdr


In [69]:
corr_THAL.to_csv('FC_corr_fig5_THAL_mc.csv')
pval_THAL.to_csv('FC_corr_pval_fig5_THAL_mc.csv')
fdr_THAL.to_csv('FC_corr_fdr_filtered_fig5_THAL_mc.csv')
#fdr_THAL.to_csv('FC_corr_fdr_BY_filtered_fig5_THAL_mc.csv')

## **MOTnetDL**

In [70]:
#Example subset from figure 6
subset6 = ['CT','Gfactor','Stand_PRS_IQ','fluid_intelligence_score_all','DUP17p12','SA',
           'Stand_PRS_SA','Stand_PRS_ASD','DUP1q21_1','DUP16p11_2','DEL16p11_2','Neuroticism',
           'Stand_PRS_MDD','DEL22q11_2','BIP','SZ','DEL15q11_2','Stand_PRS_SCZ','ASD',
          'DUP22q11_2','DEL1q21_1']

corr_MOT,pval_MOT,fdr_MOT = make_matrices(df_MOT,subset6,fdr='fdr_filtered')

<ipython-input-2-91cc07d6591e>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['fdr_filtered'] = fdr


In [71]:
corr_MOT.to_csv('FC_corr_fig6_MOT_mc.csv')
pval_MOT.to_csv('FC_corr_pval_fig6_MOT_mc.csv')
fdr_MOT.to_csv('FC_corr_fdr_filtered_fig6_MOT_mc.csv')
#fdr_MOT.to_csv('FC_corr_fdr_BY_filtered_fig6_MOT_mc.csv')

### How many correlations significant and survive FDR?
Whole connectome

In [79]:
df21 = df[(df['pair0'].isin(subset))&(df['pair1'].isin(subset))]
rej,fdr21,_,_ = multipletests(df21['pval'],method='fdr_bh')

print('all:')
print('Survives FDR: ',np.sum(rej),'/210')
print('p<0.05: ',df21[df21['pval']<0.05].shape[0],'/210')

all:
Survives FDR:  35 /210
p<0.05:  82 /210


### THAL

In [80]:
df21_THAL = df_THAL[(df_THAL['pair0'].isin(subset5))&(df['pair1'].isin(subset5))]
rej_THAL,fdr21_THAL,_,_ = multipletests(df21_THAL['pval'],method='fdr_bh')

print('thal:')
print('Survives FDR: ',np.sum(rej_THAL),'/210')
print('p<0.05: ',df21_THAL[df21_THAL['pval']<0.05].shape[0],'/210')

thal:
Survives FDR:  4 /210
p<0.05:  35 /210


In [81]:
df21_MOT = df_MOT[(df_MOT['pair0'].isin(subset6))&(df['pair1'].isin(subset6))]
rej_MOT,fdr21_MOT,_,_ = multipletests(df21_MOT['pval'],method='fdr_bh')

print('mot:')
print('Survives FDR: ',np.sum(rej_MOT),'/210')
print('p<0.05: ',df21_MOT[df21_MOT['pval']<0.05].shape[0],'/210')

mot:
Survives FDR:  1 /210
p<0.05:  62 /210
